In [24]:
import numpy as np
import itertools
import time

from typing import Tuple, Set, List, Union
from scipy.sparse import linalg as spl
from numpy import linalg as LA 

In [25]:
# Create sample data

N = 1000
D= 50

A = np.random.randn(N,D)
# artificially make some similar to others
A[5] = A[99] +np.random.randn(D)*0.05
A[20] = A[85] +np.random.randn(D)*0.15
A[13] = A[19] +np.random.randn(D)*0.25
A[56] = A[71] +np.random.randn(D)*0.5
A[45] = A[49] +np.random.randn(D)*0.66

In [27]:
def cossim(u,v):
    norm = np.linalg.norm(u)*np.linalg.norm(v)
    cosine = u@v/norm
    ang = np.arccos(cosine)
    return 1-ang/np.pi

# Brute-force

$N$ docs

Time Complexity : $O(N^2)$

Space Complexity : $O(1)$

In [28]:
true_pairs_dict = {}

thresh = 0.8

start = time.time()
for (i,u),(j,v) in itertools.combinations([(i,x) for i,x in enumerate(A)],2):
    val = cossim(u,v)
    if val > thresh:
        true_pairs_dict[(i,j)] = val
t_brute = time.time()-start

# save just the keys without the values. Easier to compare later to LSH
true_pairs = set(true_pairs_dict.keys())

print(f"Brute force calculation time: {t_brute:.3f}")
print(f"Discovered pairs:")
for k, v in true_pairs_dict.items():
    print(f"Pair: {k},\tSimilarity: {v:.2f}.")

Brute force calculation time: 4.750
Discovered pairs:
Pair: (5, 99),	Similarity: 0.99.
Pair: (13, 19),	Similarity: 0.92.
Pair: (20, 85),	Similarity: 0.95.
Pair: (56, 71),	Similarity: 0.86.


# LSH


$N$ docs,
$D$ features,
$K$ hashes buckets, 

Time Complexity : $O(N^2)$

Space Complexity :

(depends on your implementation)

1. hash function : $O(DK)$
2. hashed values : $O(K)$ 

In [29]:
b, r = 50, 18

n = b*r
print(f"Transition probability: {(1/b)**(1/r):.2f}")

Transition probability: 0.80


In [30]:
class RandomProjectionLSH():
    
    def __init__(self,
                n_signature : int,
                n_input_dimension : int,
                b : int,
                r : int,
                threshold = float,
                seed = int):
        """
        Consturct Random Projection LSH to find the candidate pair
        
        """
        assert n_signature == b * r, 'signatures should be bands (b) * rows per band (r)'
        np.random.seed(seed)
        
        self.n_input_dimension = n_input_dimension
        self.n_signature = n_signature
        self.threshold = threshold
        self.b = b
        self.r = r
        # create a gaussion distributed matrix
        self.hyperplanes = np.random.randn(n_input_dimension, n_signature)
        self.signature = None
        self.n_observation = None
    
    def fit(self, input_matrix : np.ndarray):
        """
        input_matrix (np.ndarray) - (n_observations, n_input_dimension)
        
        Hint:
        
        Use input_matrix.dot(self.hyperplane) instead of 
        np.dot(input_matrix, self.hyperplane) due to some implementation details
        https://stackoverflow.com/questions/33817189/why-is-vector-dot-product-slower-with-scipys-sparse-csr-matrix-than-numpys-den
        
        """
        self.n_observation = input_matrix.shape[0]
        self.input_matrix = input_matrix
        # build signature , shape (n_observation, n_signature)
        self.signature = input_matrix.dot(self.hyperplanes)
        self.signature = self.signature >= 0
        self.signature = np.reshape(self.signature,
                                    (self.n_observation, self.b, self.r))
        self.hashed_values = self._generate_hash()
        
    def find_candidates(self, dtype='dense') -> Tuple[Set, int]:
        """
        Apply OR logic
        
        In each band
        check is there other observation in the same bucket
        if yes, there is a cindidate
            output : 
            
            duplicates : candidate pair by id
            n_candidates : for all ids, number of candidates
        """
        if dtype == 'dense':
            cos_sim = self._consine_similarity_dense
        elif dtype == 'sparse':
            cos_sim = self._cosine_similarity_sparse
        
        ##### Apply OR Logic
        candidates = []
        for i in range(self.b):
            un_values, counts = np.unique(self.hashed_values[:, i],
                                          return_counts=True) # get unique integers and their count
            
            non_unique_values = un_values[counts > 1] # identify integers which appear more than once
            
            ####### Might be at least 1
            for val in non_unique_values: # store duplicate integers as candidates
                candidates.append(np.where(
                    self.hashed_values[:, i] == val)[0])
        
        n_candidates = len(candidates)

        # get cosine distance for every candidate
        dist = [cos_sim(self.input_matrix, cand[0], cand[1])
                for cand 
                in candidates] # get the distance for all candidates

        # get duplicates
        duplicates = set(
            (candidates[i][0], candidates[i][1], dist[i])
            for i in range(len(dist)) 
            if dist[i] >= self.threshold) 

        return duplicates, n_candidates
    
    def _consine_similarity_dense(self, X, i, j) -> float:
        
        return np.dot(X[i], X[j]) / (LA.norm(X[i]) * LA.norm(X[j]))
    
    def _cosine_similarity_sparse(self, X, i, j) -> float:
        """
        This function is for sparse vector calculations
        """
        
        i_norm = spl.norm(X[i])
        j_norm = spl.norm(X[j])
        ij_dot = X[i].dot(X[j].T)[0, 0]

        return ij_dot/(i_norm*j_norm)
    
    def _generate_hash(self) -> np.ndarray:
        """
        Apply and logic for rows per band
        
        Hash all of element in 
        the signature matrix(n_observations, b, r)
        into bucket by element-wise multify between
        
        signature and bucket_id
        
        output : 
            signature_matrix (band) shape : (n_oberservation, b)
        """
        # AND hashing
        vals = 2**np.repeat(
            [np.repeat([np.arange(self.r)], self.b, axis=0)],
            self.n_observation,
            axis=0)
        
        # elementwise product
        # construct hashed bucket
        hashed_values = np.multiply(self.signature, vals)
        hashed_values = np.sum(hashed_values, axis=2)
        return hashed_values

In [31]:
start = time.time()

rplsh = RandomProjectionLSH(
    n_signature=900,
    n_input_dimension=50,
    b=50,
    r=18,
    threshold = 0.75,
    seed=42
)

rplsh.fit(A)

lsh_similar, n_lsh_similar = rplsh.find_candidates()

t_lsh = time.time() - start

for pair in lsh_similar:
    _from, _to, sim = pair
    print(f'Pair :({_from} {_to}), Similarity: {sim}')

print()
print(f"LSH calculation time: {t_lsh:.3f}")
print(f'speed up {(t_brute / t_lsh):.1f}x')

Pair :(5 99), Similarity: 0.9990037491636654
Pair :(13 19), Similarity: 0.968380463048351
Pair :(20 85), Similarity: 0.9892354755140065
Pair :(56 71), Similarity: 0.900335887084203

LSH calculation time: 0.032
speed up 148.3x


In [32]:
# hashes

print(rplsh.hashed_values.shape)
print(rplsh.hashed_values)

(1000, 50)
[[175831 174638  66712 ...  11154 252485 125380]
 [127321     14 172531 ...  67608  38625 133278]
 [153253 260714 251782 ...  80584 242572 222939]
 ...
 [159265 151458  95319 ... 105137  66207 260123]
 [137051 256712 179063 ... 196550 126953 213781]
 [ 47401 183253  85981 ...  24836  63846 104611]]
